In [1]:
import os

In [2]:
%pwd

'e:\\MLOps\\datascienceproject\\research'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_dir_file: Path
    unzip_dir: Path


In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_dir_file=config.local_dir_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [10]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    #Downloading the Zip file 
    def download_file(self):
        if not os.path.exists(self.config.local_dir_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_dir_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info("File already exists.")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_dir_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2025-05-06 13:08:10,598: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-05-06 13:08:10,599: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-05-06 13:08:10,600: INFO: common: yaml file: schema.yaml loaded successfully.]
[2025-05-06 13:08:10,601: INFO: common: Created directory at: artifacts]
[2025-05-06 13:08:10,606: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-05-06 13:08:11,187: INFO: 154922627: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D0CD:BB6B1:175304:19A0D9:6819ED9C
Accept-Ranges: bytes
Date: Tue,